In [5]:
import ipywidgets as widgets
from IPython.display import display, clear_output
from diceroll import dice_roll_toolbox  # Import the dice_roll_toolbox class

# Create widgets for dice configuration
dice_face_slider = widgets.SelectionSlider(
    options=[4, 6, 8, 10, 12, 20, 100],
    value=6,  # Default value
    description='Dice Faces:',
    style={'description_width': 'initial'},
    continuous_update=False
)

addon_slider = widgets.IntSlider(
    value=1,
    min=1,
    max=20,
    step=1,
    description='Addon:',
    style={'description_width': 'initial'},
    continuous_update=False
)

dice_quantity_slider = widgets.IntSlider(
    value=1,
    min=1,
    max=50,
    step=1,
    description='Dice Quantity:',
    style={'description_width': 'initial'},
    continuous_update=False
)

# Create the Update button
update_button = widgets.Button(
    description='Update',
    button_style='primary',
    tooltip='Click to update plot',
    icon='refresh'
)

# Create Output widgets: one for the static plot and another for the dynamic y_slider
output_plot = widgets.Output()
output_slider = widgets.Output()

# Function to update and display the static plot when Update button is pressed
def update_plot(change=None):
    global y_slider  # Make y_slider global so it can be accessed and modified

    with output_plot:
        clear_output(wait=True)
        dice_faces = dice_face_slider.value
        addon = addon_slider.value
        dice_quantity = dice_quantity_slider.value
        
        dice_prompt = f"{dice_quantity}d{dice_faces}+{addon}"
        dice_roll = dice_roll_toolbox(dice_prompt)

        # Plot the updated results
        dice_roll.plot_values()

    # Delete the existing y_slider and create a new one
    with output_slider:
        clear_output(wait=True)  # Clear the previous slider
        # Recreate y_slider with updated properties
        y_slider = widgets.IntSlider(
            value=round(dice_roll.mean_outcome),  # Set default value to mean_value
            min=dice_roll.min_outcome,
            max=dice_roll.max_outcome,
            step=1,
            description=r'$\text{Threshold, }y=$',
            style={'description_width': 'initial'},
            continuous_update=True,
            layout=widgets.Layout(width='1100px')  # Set the slider width to 1100px
        )

        # Attach the update function to the new y_slider value change event
        y_slider.observe(update_slider_output, names='value')

        # Display the new y_slider
        display(y_slider)

# Function to update the dynamic y_slider output
def update_slider_output(change=None):
    with output_slider:
        clear_output(wait=True)
        # Display the slider
        display(y_slider)
        
        # Output the selected numerical results based on y_slider value
        dice_faces = dice_face_slider.value
        addon = addon_slider.value
        dice_quantity = dice_quantity_slider.value
        
        dice_prompt = f"{dice_quantity}d{dice_faces}+{addon}"
        dice_roll = dice_roll_toolbox(dice_prompt)
        dice_roll.output_select_numerical_results(y_slider.value)

# Attach the update function to the Update button click event
update_button.on_click(update_plot)

# Display the main sliders and the Update button
display(dice_quantity_slider)
display(dice_face_slider)
display(addon_slider)
display(update_button)

# Display the output areas
display(output_plot)
display(output_slider)

# Initial call to display the plot and setup the slider
update_plot()


IntSlider(value=1, continuous_update=False, description='Dice Quantity:', max=50, min=1, style=SliderStyle(des…

SelectionSlider(continuous_update=False, description='Dice Faces:', index=1, options=(4, 6, 8, 10, 12, 20, 100…

IntSlider(value=1, continuous_update=False, description='Addon:', max=20, min=1, style=SliderStyle(description…

Button(button_style='primary', description='Update', icon='refresh', style=ButtonStyle(), tooltip='Click to up…

Output()

Output()